In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

matplotlib.use("pgf")
matplotlib.rcParams.update({
    'pgf.texsystem': 'pdflatex',
    'text.usetex': True,
})
%config InlineBackend.figure_format = 'retina'

In [2]:
def bayes_utilities(betas, lams, rho, tau):
    Gamma = betas.sum()
    gammas = Gamma - betas
    us = (rho - betas)**2 / rho**2 / (Gamma + tau) \
        - 1 / tau - betas / rho**2\
        + lams * gammas / (rho + tau) / (gammas + rho + tau)
    return us

def dp_utilities(betas, lams, rho, kappa, eps=1e-8):
    Gamma = betas.sum()
    gammas = Gamma - betas
    us = -kappa**2 * rho * betas / (rho - betas + eps) + \
        lams * gammas / rho / (rho + gammas)
    return us

In [3]:
def grid_search_best_beta(
    weights_fn, utility_fn, lams, rho, grid_size=100, eps=1e-8, **params
):
    weights = weights_fn(lams, rho, **params)
    init_b = rho / (weights.max() + eps)
    for b in np.linspace(init_b, 0, grid_size):
        betas = weights * b
        if (utility_fn(betas, lams, rho, **params) >= 0).all():
            return betas
    return weights * 0

In [4]:
def simple_weights(lams, rho, **kwargs):
    return np.ones_like(lams)

def xi_weights(lams, rho, tau, **kwargs):
    nominator = lams / (rho + tau)**2 - 1 / tau**2
    nominator = nominator.clip(0)
    denominator = nominator + (rho + tau)**2 / tau**2 / rho**2
    return nominator / denominator

def zeta_weights(lams, rho, kappa, **kwargs):
    zetas = lams / (lams + kappa**2 * rho**2)
    return zetas

In [5]:
def compare_accs_experiment(
    tries, rho,
    lams_fn,
    lams_params,
    utility_fn,
    simple_weights_fn,
    our_weigths_fn,
    utility_params
):
    simple_Gammas = np.empty(tries)
    our_Gammas = np.empty(tries)
    for t in range(tries):
        lams = lams_fn(**lams_params)
        betas = grid_search_best_beta(
            simple_weights_fn, utility_fn, lams, rho, **utility_params
        )
        simple_Gammas[t] = betas.sum()
        betas = grid_search_best_beta(
            our_weigths_fn, utility_fn, lams, rho, **utility_params
        )
        our_Gammas[t] = betas.sum()
    return simple_Gammas, our_Gammas

In [6]:
rng = np.random.default_rng(seed=179)

In [7]:
#common parameters
N = 5
n = 100
sigma = 10.0
rho = n / sigma**2

In [8]:
#dp parameters
B = 20.0
kappa = (2 * np.log(1.25 * n**2))**0.5 * B / n

dp_params = {"kappa": kappa}

tries = 300
lams_fn = lambda **x: rng.lognormal(**x)

grid_size = 9
our_results = np.empty(grid_size)
our_std = np.empty(grid_size)
simple_results = np.empty(grid_size)
simple_std = np.empty(grid_size)
grid = np.linspace(-2.5, 3.5, grid_size)

for t, mean in enumerate(tqdm(grid)):
    lams_params = {"mean": mean, "sigma": 1.0, "size": N}
    simple_Gammas, our_Gammas = compare_accs_experiment(
        tries, rho,
        lams_fn, lams_params,
        dp_utilities, simple_weights, zeta_weights, dp_params
    )
    our_Gammas /= N * rho
    simple_Gammas /= N * rho
    our_results[t] = our_Gammas.mean()
    our_std[t] = our_Gammas.std(ddof=1) / tries**0.5
    simple_results[t] = simple_Gammas.mean()
    simple_std[t] = simple_Gammas.std(ddof=1) / tries**0.5
results = [(grid, our_results, our_std, simple_results, simple_std)]

  0%|          | 0/9 [00:00<?, ?it/s]

In [9]:
#bayes parameters
tau = 1.0

bayes_params = {"tau": tau}

tries = 300
lams_fn = lambda **x: rng.lognormal(**x)

grid_size = 9
our_results = np.empty(grid_size)
our_std = np.empty(grid_size)
simple_results = np.empty(grid_size)
simple_std = np.empty(grid_size)
grid = np.linspace(1.5, 4.5, grid_size)

for t, mean in enumerate(tqdm(grid)):
    lams_params = {"mean": mean, "sigma": 1.0, "size": N}
    simple_Gammas, our_Gammas = compare_accs_experiment(
        tries, rho,
        lams_fn, lams_params,
        bayes_utilities, simple_weights, xi_weights, bayes_params
    )
    our_Gammas /= N * rho
    simple_Gammas /= N * rho
    our_results[t] = our_Gammas.mean()
    our_std[t] = our_Gammas.std(ddof=1) / tries**0.5
    simple_results[t] = simple_Gammas.mean()
    simple_std[t] = simple_Gammas.std(ddof=1) / tries**0.5
results.append((grid, our_results, our_std, simple_results, simple_std))

  0%|          | 0/9 [00:00<?, ?it/s]

In [ ]:
plt.close()

fig, axes = plt.subplots(2, 1)

fig.set_size_inches(w=3.55, h=4.5)

ax = axes[0]
grid, our_results, our_std, simple_results, simple_std = results[0]

ax.errorbar(grid, our_results, yerr=our_std,
            label="personalized", linewidth=0.7)
ax.errorbar(grid, simple_results, yerr=simple_std,
            label="symmetric", linewidth=0.7)
#ax.plot(grid, our_results, label="personalized")
#ax.plot(grid, simple_results, label="symmetric")
size = 8
plt.setp(ax.spines.values(), linewidth=0.5)
ax.tick_params(width=0.5)
ax.legend(fontsize=size)
#ax.set_xlabel(
#    r"Mean Importance of Accuracy ($\mathbf{E}(\ln \lambda)$)",
#    fontsize=size)
ax.set_ylabel(
    r"Protocol Effectiveness ($\Gamma / \Gamma^*$)",
    labelpad=1, fontsize=size
)
ax.tick_params(axis='both', labelsize=7)

ax = axes[1]
grid, our_results, our_std, simple_results, simple_std = results[1]

ax.errorbar(grid, our_results, yerr=our_std,
            label="personalized", linewidth=0.7)
ax.errorbar(grid, simple_results, yerr=simple_std,
            label="symmetric", linewidth=0.7)
#ax.plot(grid, our_results, label="personalized")
#ax.plot(grid, simple_results, label="symmetric")
plt.setp(ax.spines.values(), linewidth=0.5)
ax.tick_params(width=0.5)
ax.legend(fontsize=size)
ax.set_xlabel(
    r"Mean Accuracy Importance ($\mathbf{E}(\ln \lambda)$)",
    labelpad=1, fontsize=size)
ax.set_ylabel(
    r"Protocol Effectiveness ($\Gamma / \Gamma^*$)",
    labelpad=1, fontsize=size
)
ax.tick_params(axis='both', labelsize=7)
plt.gcf().tight_layout()
plt.savefig('exp-mean.pgf', bbox_inches='tight', pad_inches=0)